In [1]:
import pandas as pd
import re
from ipywidgets import Textarea, Dropdown, Button, Output, IntText
from IPython.display import display


In [2]:
payroll_data = {
    "employee_id": [1001, 1002, 1003],
    "name": ["Rahul", "Anita", "Karthik"],
    "department": ["IT", "Finance", "HR"],
    "role": ["EMPLOYEE", "EMPLOYEE", "HR"],
    "basic_salary": [30000, 25000, 45000],
    "hra": [12000, 10000, 18000],
    "other_allowances": [4000, 3000, 5000],
    "gross_salary": [46000, 38000, 68000],
    "pf_applicable": ["Yes", "Yes", "Yes"],
    "esi_applicable": ["No", "Yes", "No"],
    "professional_tax": [200, 200, 200]
}

df = pd.DataFrame(payroll_data)
df


,employee_id,name,department,role,basic_salary,hra,other_allowances,gross_salary,pf_applicable,esi_applicable,professional_tax
0,1001,Rahul,IT,EMPLOYEE,30000,12000,4000,46000,Yes,No,200
1,1002,Anita,Finance,EMPLOYEE,25000,10000,3000,38000,Yes,Yes,200
2,1003,Karthik,HR,HR,45000,18000,5000,68000,Yes,No,200


In [3]:
df.to_csv("sample_payroll_dataset.csv", index=False)
PAYROLL_RULES = """
PF: 12% of Basic Salary (EPF Act, India)
ESI: 0.75% employee contribution if Gross Salary < ₹21,000
Professional Tax: ₹200 per month (state dependent)
Income Tax: As per government tax slabs
Bonus: Fully taxable
Unpaid Leave: Reduces salary proportionally
"""


In [4]:
SYSTEM_PROMPT = """
You are a Payroll AI Copilot.

Rules:
- Answer ONLY using payroll rules and provided context.
- If data is missing, say: "I don't have enough information."
- Never guess legal or financial values.
- Explain deductions legally and ethically.
- Redact sensitive data.
- Enforce role-based access.
"""


In [5]:
ROLE_PROMPTS = {
    "HR": """
You are responding to an HR user.
Use formal, compliance-focused language.
""",

    "EMPLOYEE": """
You are responding to an employee.
Use simple, clear language.
Do not expose internal payroll policies.
"""
}


In [6]:
HR_ONLY_INTENTS = {
    "STATUTORY",
    "COMPLIANCE",
    "PAYROLL_ERROR",
    "SALARY_REVISION"
}


In [7]:
def detect_intent(query):
    q = query.lower()

    # Common
    if "next year" in q or "future" in q:
        return "FUTURE_PREDICTION"
    if "pf" in q:
        return "PF"
    if "esi" in q:
        return "ESI"
    if "net salary" in q or "lower" in q:
        return "NET_SALARY"
    if "bonus" in q:
        return "BONUS"
    if "tax" in q:
        return "TAX"

    # HR-specific
    if "statutory" in q:
        return "STATUTORY"
    if "compliance" in q:
        return "COMPLIANCE"
    if "payroll error" in q or "error" in q:
        return "PAYROLL_ERROR"
    if "salary revision" in q:
        return "SALARY_REVISION"

    return "UNKNOWN"


In [8]:
def get_employee_data(emp_id):
    emp = df[df["employee_id"] == emp_id]
    if emp.empty:
        return None
    return emp.to_dict(orient="records")[0]


In [9]:
def build_context(emp_id):
    emp = get_employee_data(emp_id)

    if emp is None:
        return "Employee record not found."

    return f"""
Employee ID: {emp['employee_id']}
Department: {emp['department']}
Basic Salary: ₹{emp['basic_salary']}
Gross Salary: ₹{emp['gross_salary']}
PF Applicable: {emp['pf_applicable']}
ESI Applicable: {emp['esi_applicable']}

Payroll Rules:
{PAYROLL_RULES}
"""


In [10]:
def redact_sensitive(text):
    patterns = {
        "PAN": r"[A-Z]{5}[0-9]{4}[A-Z]",
        "Aadhaar": r"\b\d{12}\b",
        "Bank": r"\b\d{9,18}\b"
    }

    for label, pattern in patterns.items():
        text = re.sub(pattern, f"[REDACTED {label}]", text)

    return text


In [11]:
def hallucination_guard(response):
    risky_words = ["maybe", "probably", "might", "around", "approx"]

    for word in risky_words:
        if word in response.lower():
            return "⚠️ Cannot answer due to insufficient confirmed data."

    return response


In [12]:
def payroll_ai_copilot(query, role, emp_id):
    emp = get_employee_data(emp_id)

    if emp is None:
        return "Employee record not found."

    intent = detect_intent(query)

    # Block HR-only questions for employees
    if role == "EMPLOYEE" and intent in HR_ONLY_INTENTS:
        return "You are not authorized to access this information."

    if role == "HR":
        response = generate_hr_answer(intent, emp)
    else:
        response = generate_employee_answer(intent, emp)

    response = redact_sensitive(response)
    response = hallucination_guard(response)

    return response


In [13]:
def generate_employee_answer(intent, emp):
    if intent == "FUTURE_PREDICTION":
        return "I don't have enough information to predict future salary."

    if intent == "PF":
        return (
            f"PF is deducted at 12% of basic salary. "
            f"Your basic salary is ₹{emp['basic_salary']}."
        )

    if intent == "ESI":
        return (
            "ESI is applicable to you." if emp["esi_applicable"] == "Yes"
            else "ESI is not applicable to you based on your salary."
        )

    if intent == "NET_SALARY":
        return (
            "Your net salary may be lower due to PF, professional tax, "
            "or unpaid leave deductions."
        )

    if intent == "BONUS":
        return "Bonuses are fully taxable as per income tax rules."

    if intent == "TAX":
        return "Income tax is calculated as per government tax slabs."

    return "I don't have enough information to answer this question."


In [14]:
def generate_hr_answer(intent, emp):
    if intent == "FUTURE_PREDICTION":
        return (
            "Future salary projections cannot be confirmed without "
            "official appraisal and approval records."
        )

    if intent == "PF":
        return (
            "Provident Fund is a statutory deduction under the EPF Act. "
            "The employer must deduct 12% of basic salary and remit it to EPFO."
        )

    if intent == "ESI":
        return (
            "ESI applicability depends on statutory wage limits. "
            "Employers must ensure timely ESI contributions for eligible employees."
        )

    if intent == "STATUTORY":
        return (
            "Statutory deductions include PF, ESI, Professional Tax, "
            "and Income Tax as mandated by law."
        )

    if intent == "COMPLIANCE":
        return (
            "Payroll compliance requires timely statutory remittances, "
            "accurate payroll records, and audit readiness."
        )

    if intent == "PAYROLL_ERROR":
        return (
            "Payroll errors must be corrected through an auditable adjustment "
            "process with documented approvals."
        )

    if intent == "SALARY_REVISION":
        return (
            "Salary revisions require formal approval, effective-date control, "
            "and payroll recalculation with audit trails."
        )

    return "Insufficient information to provide an HR-level response."


In [15]:
emp_id_box = IntText(
    value=1001,
    description="Emp ID:"
)

query_box = Textarea(
    placeholder="Type your payroll question here...",
    layout={'width': '600px', 'height': '100px'}
)

role_dropdown = Dropdown(
    options=['EMPLOYEE', 'HR'],
    value='EMPLOYEE',
    description='Role:'
)

submit_button = Button(
    description="Ask Payroll AI",
    button_style='success'
)

output_area = Output()


In [16]:
def on_submit_clicked(b):
    with output_area:
        output_area.clear_output()
        response = payroll_ai_copilot(
            query_box.value,
            role_dropdown.value,
            emp_id_box.value
        )
        print(" Payroll AI Response:\n")
        print(response)

submit_button.on_click(on_submit_clicked)

display(emp_id_box, query_box, role_dropdown, submit_button, output_area)


IntText(value=1001, description='Emp ID:')

Textarea(value='', layout=Layout(height='100px', width='600px'), placeholder='Type your payroll question here.…

Dropdown(description='Role:', options=('EMPLOYEE', 'HR'), value='EMPLOYEE')

Button(button_style='success', description='Ask Payroll AI', style=ButtonStyle())

Output()

SAMPLE EMPLOYEE PROMPTS

1. Why is PF deducted from my salary every month?

2. Is ESI applicable to me based on my salary?

3. My net salary is lower this month. Can you explain why?

4. Is the annual bonus taxable or tax-free?

5. What will my salary be next year?


SAMPLE HR PROMPTS

1. Explain statutory deductions applicable for this employee.

2. What payroll compliance rules should the company follow?

3. How should payroll errors be handled ethically and legally?

4. Explain the salary revision process from an HR perspective.

5. What audit controls are required in payroll processing?
